In [4]:
import json
import os
import sys
from collections import defaultdict
from datetime import datetime

import requests
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup

from marketAPI import AlphaVantage

%matplotlib inline

import pickle

In [101]:
today = pd.Timestamp.today()

date_str = today.date().strftime("%Y%m%d")
timestamp = today.timestamp()

In [59]:
active_df = pd.read_html("https://finance.yahoo.com/most-active")[0]

In [ ]:
https://robinhood.com/collections/100-most-popular

In [60]:
def shortConvert(cap):
    multiplier = cap[-1]
    number = {'M': 1e6, 'B': 1e9, 'T': 1e12}
    
    return float(cap[:-1]) * number.get(multiplier)

In [61]:
active_df['% Change'] = active_df['% Change'].apply(lambda x: float(x.strip("+%")))
active_df.loc[:, ['Market Cap', 'Volume']] = active_df.loc[:, ['Market Cap', 'Volume']].applymap(shortConvert)

In [65]:
top_gainers = active_df.sort_values('% Change').iloc[-3:]
top_losers = active_df.sort_values('% Change').iloc[:3]

In [69]:
top_gainers

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
34,DKNG,DraftKings Inc.,35.64,3.08,9.46,18160000.0,11.919M,1.261100e+10,NaN,NaN
19,DELL,Dell Technologies Inc.,59.10,6.42,12.19,24709000.0,2.744M,4.378200e+10,10.13,NaN
7,SPCE,"Virgin Galactic Holdings, Inc.",21.07,2.52,13.58,51832000.0,15.211M,4.433000e+09,NaN,NaN


In [67]:
# active_dict = active_df.set_index("Symbol").T.to_dict()
# active_order = active_df.Symbol.tolist()
# with open(f"activeTickers_{date_str}.pkl", "wb") as f:
#     pickle.dump(active_dict, f)

In [70]:
symbol = "SPCE"
interval="5min"

av = AlphaVantage()
intraday = av.intradayStock(symbol, interval=interval, size="compact")
daily = av.dailyAdjStock(symbol, size='full')

In [81]:
intradayData = intraday[f'Time Series ({interval})']
intradayDict = defaultdict(dict)
for k, v in intradayData.items():
    date_key = datetime.strptime(k, "%Y-%m-%d %H:%M:%S")
    intradayDict[date_key]['close'] = v['4. close']
    intradayDict[date_key]['volume'] = v['5. volume']

dailyData = daily[f'Time Series (Daily)']
dailyDict = defaultdict(dict)
for k, v in dailyData.items():
    date_key = datetime.strptime(k, "%Y-%m-%d")
    dailyDict[date_key]['close'] = v['5. adjusted close']
    dailyDict[date_key]['volume'] = v['6. volume']

In [103]:

if not os.path.exists(f'data_{date_str}'):
    os.mkdir(f'data_{date_str}')
    


In [110]:

(
    pd.DataFrame(dailyDict).T
    .iloc[:200]
    .reset_index()
    .rename(columns={'index': 'date'})
    .to_csv(f'data_{date_str}/{symbol}_daily_{timestamp}.csv', index=False)
)

In [105]:
(
    pd.DataFrame(intradayDict).T
    .reset_index()
    .rename(columns={'index': 'time'})
    .to_csv(f'data_{date_str}/{symbol}_intraday_{timestamp}.csv', index=False)
)